In [1]:
import numpy as np
import pandas as pd
from gensim.models import word2vec
import nltk
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

C:\Users\PDudukin\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [21]:
df = pd.concat([
    pd.read_csv('./data/train.zip',index_col='id'),
    pd.read_csv('./data/test.zip',index_col='id')
    ],axis=0)
df['part'] = df['author'].isnull().apply(lambda x: 'test' if x else 'train')
print('Количество примеров:',len(df),', доля обучающей выборки:',"{0:.2f}%".format(100*np.mean(df['part']=='train')))
df.sample(5)

Количество примеров: 27971 , доля обучающей выборки: 70.00%


,author,text,part
id,,,
id07574,MWS,"All was again silent, but his words rang in my...",train
id26064,MWS,Confinement and hardship had undermined his he...,train
id06305,EAP,The whole company at once started to their fee...,train
id16121,NaN,The only egress was the aperture in the floor ...,test
id17739,MWS,"And since in the wide world thou remainest, wh...",train


In [22]:
tkn = Tokenizer()
tkn.fit_on_texts(df.text)
df['tokens'] = tkn.texts_to_sequences(df.text)
max_text_len = max(df.text.apply(len))
print('Максимальная длина текста:',max_text_len,' слов')
df['tokens'] = list(sequence.pad_sequences(df['tokens'].values, maxlen=max_text_len))

Максимальная длина текста: 4663  слов


In [64]:
len(tkn.word_index)

29451

In [26]:
test_x = df[df['part']=='test']['tokens'].apply(list).tolist()
train = df[df['part']=='train']
train_x,valid_x,train_y,valid_y = train_test_split(train['tokens'].apply(list).tolist(),pd.get_dummies(train['author']))

In [33]:
max_features = 30000
batch_size = 200

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32,return_sequences=True))
model.add(LSTM(16, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              #optimizer='adam',
              optimizer=RMSprop(),
              metrics=['categorical_accuracy'])

print('Train...')
model.fit(
    train_x,
    train_y.values,
    batch_size=batch_size,
    validation_data=(valid_x, valid_y.values),
    epochs=1
)

Build model...
Train...
Train on 14684 samples, validate on 4895 samples
Epoch 1/1
14684/14684 [==============================] - 646s 44ms/step - loss: 0.9983 - categorical_accuracy: 0.4938 - val_loss: 0.8875 - val_categorical_accuracy: 0.5608


NameError: name 'score' is not defined

In [65]:
print('Train...')
model.fit(
    train_x,
    train_y.values,
    batch_size=batch_size,
    validation_data=(valid_x, valid_y.values),
    epochs=1
)

Train...
Train on 14684 samples, validate on 4895 samples
Epoch 1/1
14684/14684 [==============================] - 717s 49ms/step - loss: 0.3309 - categorical_accuracy: 0.8786 - val_loss: 0.5372 - val_categorical_accuracy: 0.7906


In [44]:
score, acc = model.evaluate(valid_x, valid_y.values,batch_size=batch_size,)
print('Test score:', score)
print('Test accuracy:', acc)

4895/4895 [==============================] - 37s 8ms/step
Test score: 0.583391560415
Test accuracy: 0.769560770738


In [66]:
test = df[df['part']=='test']#
test_x = test['tokens'].apply(list).tolist()
pred = model.predict_proba(test_x)
for i,e in enumerate(cols):
    test[e] = pred[:,i]
test

8392/8392 [==============================] - 237s 28ms/step


/Users/sermolov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,author,text,part,tokens,EAP,HPL,MWS
id,,,,,,,
id02310,NaN,"Still, as I urged our leaving Ireland with suc...",test,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.070467,0.025933,0.903599
id24541,NaN,"If a fire wanted fanning, it could readily be ...",test,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.994872,0.002130,0.002998
id00134,NaN,And when they had broken down the frail door t...,test,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.155731,0.438179,0.406090
id27757,NaN,While I was thinking how I should possibly man...,test,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.905869,0.017362,0.076769
id04081,NaN,I am not sure to what limit his knowledge may ...,test,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.905038,0.021902,0.073060
id27337,NaN,"""The thick and peculiar mist, or smoke, which ...",test,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.974823,0.007937,0.017240
id24265,NaN,"That which is not matter, is not at all unless...",test,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.955759,0.015006,0.029235
id25917,NaN,I sought for repose although I did not hope fo...,test,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.029040,0.219524,0.751436
id04951,NaN,"Upon the fourth day of the assassination, a pa...",test,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.995064,0.002232,0.002705


In [67]:
cols = list(pd.get_dummies(train['author']).columns)
print(cols)
test['author'] = test['prediction'].apply(lambda x: cols[x])
test.sample(10)

['EAP', 'HPL', 'MWS']


KeyError: 'prediction'

In [68]:
test[cols].to_csv('./submissions/rnn_small_v02.csv')

In [2]:
model_forward = Sequential()
model_forward.add(Embedding(max_features, embedding_size, input_length=maxlen, mask_zero=True))
model_forward.add(LSTM(hidden_size, return_sequences=True))  

model_backward = Sequential()
model_backward.add(Embedding(max_features, embedding_size, input_length=maxlen, mask_zero=True))
model_backward.add(LSTM(hidden_size, return_sequences=True))  
def reverse_func(x, mask=None):
    return tf.reverse(x, [False, True, False])
model_backward.add(MaskLambda(function=reverse_func, mask_function=reverse_func))

model = Sequential()
model.add(Merge([model_forward, model_backward], mode='concat'))
model.add(TimeDistributedDense(out_size))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

NameError: name 'max_features' is not defined